# Holographic Recycling Cosmology: Results Analysis

This notebook analyzes the constraints on HRC model parameters from cosmological observations.

## Key Questions
1. Can HRC resolve the Hubble tension?
2. What parameter values are favored?
3. How does HRC compare to ΛCDM?

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path

# HRC modules
from hrc_observations import (
    ObservationalData,
    HRCPredictions,
    HRCLikelihood,
    LCDMCosmology,
    compute_bayes_factor
)

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12

## 1. Load Observational Data

In [ ]:
# Load all observational constraints
data = ObservationalData()
print(data.summary())

In [ ]:
# Visualize the Hubble tension
fig, ax = plt.subplots(figsize=(10, 4))

# Measurements
measurements = [
    ('Planck 2018', 67.4, 0.5, 'blue'),
    ('ACT DR6', 67.9, 1.5, 'royalblue'),
    ('SH0ES 2024', 73.04, 1.04, 'red'),
    ('TRGB', 69.8, 1.7, 'orange'),
    ('TDCOSMO', 73.3, 3.3, 'purple'),
]

for i, (name, h0, err, color) in enumerate(measurements):
    ax.errorbar(h0, i, xerr=err, fmt='o', color=color, 
                capsize=5, markersize=10, label=f'{name}: {h0}±{err}')

ax.axvline(67.4, color='blue', alpha=0.3, linestyle='--', lw=2)
ax.axvline(73.0, color='red', alpha=0.3, linestyle='--', lw=2)
ax.axvspan(66.9, 67.9, alpha=0.1, color='blue')
ax.axvspan(72.0, 74.1, alpha=0.1, color='red')

ax.set_xlabel('$H_0$ [km/s/Mpc]', fontsize=14)
ax.set_yticks(range(len(measurements)))
ax.set_yticklabels([m[0] for m in measurements])
ax.set_xlim(63, 78)
ax.legend(loc='upper right', fontsize=10)
ax.set_title('The Hubble Tension (~5σ)', fontsize=14)

plt.tight_layout()
plt.savefig('figures/hubble_tension_overview.png', dpi=150)
plt.show()

## 2. HRC Model Predictions

In [ ]:
# Explore how HRC parameters affect H0 measurements

# Scan ξ and φ₀
xi_range = np.linspace(-0.05, 0.05, 50)
phi_range = np.linspace(0, 0.5, 50)

H0_local_grid = np.zeros((len(phi_range), len(xi_range)))
H0_cmb_grid = np.zeros((len(phi_range), len(xi_range)))
delta_H0_grid = np.zeros((len(phi_range), len(xi_range)))

for i, phi_0 in enumerate(phi_range):
    for j, xi in enumerate(xi_range):
        params = {
            'H0_true': 70.0,
            'Omega_m': 0.315,
            'xi': xi,
            'alpha': 0.01,
            'phi_0': phi_0,
        }
        try:
            pred = HRCPredictions(params)
            H0_local_grid[i, j] = pred.H0_local()
            H0_cmb_grid[i, j] = pred.H0_cmb()
            delta_H0_grid[i, j] = H0_local_grid[i, j] - H0_cmb_grid[i, j]
        except:
            H0_local_grid[i, j] = np.nan
            H0_cmb_grid[i, j] = np.nan
            delta_H0_grid[i, j] = np.nan

In [ ]:
# Plot ΔH₀ as function of parameters
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

X, Y = np.meshgrid(xi_range, phi_range)

# Delta H0
ax = axes[0]
levels = np.linspace(-10, 10, 21)
cf = ax.contourf(X, Y, delta_H0_grid, levels=levels, cmap='RdBu_r')
plt.colorbar(cf, ax=ax, label='$\\Delta H_0$ [km/s/Mpc]')
# Mark observed tension
cs = ax.contour(X, Y, delta_H0_grid, levels=[5.6], colors='lime', linewidths=2)
ax.set_xlabel('$\\xi$')
ax.set_ylabel('$\\phi_0$')
ax.set_title('$H_0^{\\rm local} - H_0^{\\rm CMB}$')

# H0 local
ax = axes[1]
levels = np.linspace(65, 80, 16)
cf = ax.contourf(X, Y, H0_local_grid, levels=levels, cmap='Reds')
plt.colorbar(cf, ax=ax, label='$H_0^{\\rm local}$ [km/s/Mpc]')
cs = ax.contour(X, Y, H0_local_grid, levels=[73.0], colors='black', linewidths=2)
ax.set_xlabel('$\\xi$')
ax.set_ylabel('$\\phi_0$')
ax.set_title('Local $H_0$ (target: 73)')

# H0 CMB
ax = axes[2]
cf = ax.contourf(X, Y, H0_cmb_grid, levels=levels, cmap='Blues')
plt.colorbar(cf, ax=ax, label='$H_0^{\\rm CMB}$ [km/s/Mpc]')
cs = ax.contour(X, Y, H0_cmb_grid, levels=[67.4], colors='black', linewidths=2)
ax.set_xlabel('$\\xi$')
ax.set_ylabel('$\\phi_0$')
ax.set_title('CMB-inferred $H_0$ (target: 67.4)')

plt.tight_layout()
plt.savefig('figures/hrc_parameter_scan.png', dpi=150)
plt.show()

## 3. Distance-Redshift Relations

In [ ]:
# Compare HRC and ΛCDM distance predictions

# ΛCDM
lcdm = LCDMCosmology(H0=67.4, Omega_m=0.315)

# HRC with tension-resolving parameters
hrc_params = {
    'H0_true': 70.0,
    'Omega_m': 0.315,
    'xi': 0.015,
    'alpha': 0.01,
    'phi_0': 0.2,
}
hrc = HRCPredictions(hrc_params)

# Redshift range
z_array = np.logspace(-2, 0.5, 100)

# Compute distances
DL_lcdm = np.array([lcdm.luminosity_distance(z) for z in z_array])
DL_hrc = np.array([hrc.luminosity_distance(z) for z in z_array])

# Distance modulus
mu_lcdm = np.array([lcdm.distance_modulus(z) for z in z_array])
mu_hrc = np.array([hrc.distance_modulus(z) for z in z_array])

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Luminosity distance
ax = axes[0, 0]
ax.loglog(z_array, DL_lcdm, 'b-', label='ΛCDM', lw=2)
ax.loglog(z_array, DL_hrc, 'r--', label='HRC', lw=2)
ax.set_xlabel('Redshift $z$')
ax.set_ylabel('$D_L$ [Mpc]')
ax.legend()
ax.set_title('Luminosity Distance')

# Relative difference
ax = axes[0, 1]
diff = (DL_hrc - DL_lcdm) / DL_lcdm * 100
ax.semilogx(z_array, diff, 'k-', lw=2)
ax.axhline(0, color='gray', linestyle='--')
ax.set_xlabel('Redshift $z$')
ax.set_ylabel('$(D_L^{\\rm HRC} - D_L^{\\rm ΛCDM})/D_L^{\\rm ΛCDM}$ [%]')
ax.set_title('Relative Distance Difference')

# Distance modulus with SN data
ax = axes[1, 0]
ax.plot(z_array, mu_lcdm, 'b-', label='ΛCDM', lw=2)
ax.plot(z_array, mu_hrc, 'r--', label='HRC', lw=2)
# Add Pantheon+ data
sn = data.sn_data
ax.errorbar(sn.z_bins, sn.mu_bins, yerr=sn.mu_err, fmt='ko', 
            capsize=3, markersize=4, label='Pantheon+')
ax.set_xlabel('Redshift $z$')
ax.set_ylabel('Distance modulus $\\mu$')
ax.legend()
ax.set_xlim(0, 2)
ax.set_title('Distance Modulus vs Pantheon+ SNe')

# Hubble diagram residuals
ax = axes[1, 1]
mu_lcdm_at_data = np.array([lcdm.distance_modulus(z) for z in sn.z_bins])
mu_hrc_at_data = np.array([hrc.distance_modulus(z) for z in sn.z_bins])
residuals_lcdm = sn.mu_bins - mu_lcdm_at_data
residuals_hrc = sn.mu_bins - mu_hrc_at_data

ax.errorbar(sn.z_bins, residuals_lcdm, yerr=sn.mu_err, fmt='bo', 
            capsize=3, markersize=5, label='ΛCDM residuals')
ax.errorbar(sn.z_bins + 0.01, residuals_hrc, yerr=sn.mu_err, fmt='rs', 
            capsize=3, markersize=5, label='HRC residuals')
ax.axhline(0, color='gray', linestyle='--')
ax.set_xlabel('Redshift $z$')
ax.set_ylabel('$\\mu_{\\rm obs} - \\mu_{\\rm model}$')
ax.legend()
ax.set_title('Hubble Diagram Residuals')

plt.tight_layout()
plt.savefig('figures/distance_comparison.png', dpi=150)
plt.show()

## 4. BAO Comparison

In [ ]:
# Compare BAO predictions
bao = data.bao_measurements
r_d_planck = bao.r_d_planck

# Model predictions
DV_rd_lcdm = np.array([lcdm.D_V(z) / r_d_planck for z in bao.z_eff])

r_s_hrc = hrc.sound_horizon()
DV_rd_hrc = np.array([hrc.D_V(z) / r_s_hrc for z in bao.z_eff])

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ax = axes[0]
ax.errorbar(bao.z_eff, bao.DV_rd, yerr=bao.DV_rd_err, fmt='ko', 
            capsize=5, markersize=8, label='DESI BAO')
ax.plot(bao.z_eff, DV_rd_lcdm, 'b-o', label='ΛCDM', markersize=6)
ax.plot(bao.z_eff, DV_rd_hrc, 'r--s', label='HRC', markersize=6)
ax.set_xlabel('Redshift $z$')
ax.set_ylabel('$D_V / r_d$')
ax.legend()
ax.set_title('BAO Distance Scale')

ax = axes[1]
residuals_lcdm = (DV_rd_lcdm - bao.DV_rd) / bao.DV_rd_err
residuals_hrc = (DV_rd_hrc - bao.DV_rd) / bao.DV_rd_err

ax.bar(bao.z_eff - 0.02, residuals_lcdm, width=0.03, color='blue', 
       alpha=0.7, label='ΛCDM')
ax.bar(bao.z_eff + 0.02, residuals_hrc, width=0.03, color='red', 
       alpha=0.7, label='HRC')
ax.axhline(0, color='gray', linestyle='--')
ax.axhline(2, color='gray', linestyle=':', alpha=0.5)
ax.axhline(-2, color='gray', linestyle=':', alpha=0.5)
ax.set_xlabel('Redshift $z$')
ax.set_ylabel('Residual [$\\sigma$]')
ax.legend()
ax.set_title('BAO Residuals')

plt.tight_layout()
plt.savefig('figures/bao_comparison.png', dpi=150)
plt.show()

# Chi-squared
chi2_lcdm = np.sum(residuals_lcdm**2)
chi2_hrc = np.sum(residuals_hrc**2)
print(f"BAO χ² (ΛCDM): {chi2_lcdm:.2f}")
print(f"BAO χ² (HRC):  {chi2_hrc:.2f}")

## 5. Model Comparison

In [ ]:
# Compute Bayes factor
bf_result = compute_bayes_factor(data, hrc_params)

print("MODEL COMPARISON: HRC vs ΛCDM")
print("=" * 50)
print(f"Log Bayes factor: ln(B) = {bf_result['ln_B']:.2f}")
print(f"Bayes factor: B = {bf_result['B']:.3f}")
print(f"ΔBIC = {bf_result['delta_bic']:.2f}")
print(f"\nInterpretation: {bf_result['interpretation']}")

## 6. Summary and Conclusions

In [ ]:
# Summary plot
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. H0 comparison
ax = axes[0, 0]
categories = ['Planck\n(CMB)', 'SH0ES\n(Local)', 'HRC\nCMB', 'HRC\nLocal']
values = [67.4, 73.04, hrc.H0_cmb(), hrc.H0_local()]
errors = [0.5, 1.04, 1.0, 1.0]  # Approximate
colors = ['blue', 'red', 'purple', 'green']

ax.bar(categories, values, yerr=errors, capsize=5, color=colors, alpha=0.7)
ax.axhline(67.4, color='blue', linestyle='--', alpha=0.5)
ax.axhline(73.04, color='red', linestyle='--', alpha=0.5)
ax.set_ylabel('$H_0$ [km/s/Mpc]')
ax.set_title('Hubble Constant: Observations vs HRC')

# 2. Tension resolution
ax = axes[0, 1]
observed_tension = 5.6
hrc_tension = hrc.H0_local() - hrc.H0_cmb()
bars = ax.bar(['Observed', 'HRC Prediction'], [observed_tension, hrc_tension],
              color=['gray', 'green'], alpha=0.7)
ax.set_ylabel('$\\Delta H_0$ [km/s/Mpc]')
ax.set_title('Hubble Tension')
ax.axhline(observed_tension, color='red', linestyle='--', label='Target')
ax.legend()

# 3. Parameter values
ax = axes[1, 0]
param_names = ['$\\xi$', '$\\alpha$', '$\\phi_0$']
param_values = [hrc_params['xi'], hrc_params['alpha'], hrc_params['phi_0']]
ax.bar(param_names, param_values, color='teal', alpha=0.7)
ax.set_ylabel('Parameter Value')
ax.set_title('Best-fit HRC Parameters')

# 4. Model evidence
ax = axes[1, 1]
model_names = ['ΛCDM', 'HRC']
bic_values = [bf_result['bic_lcdm'], bf_result['bic_hrc']]
ax.bar(model_names, bic_values, color=['blue', 'green'], alpha=0.7)
ax.set_ylabel('BIC')
ax.set_title(f"Model Selection (ΔBIC = {bf_result['delta_bic']:.1f})")

plt.tight_layout()
plt.savefig('figures/summary_plot.png', dpi=150)
plt.show()

In [ ]:
# Print conclusions
print("\n" + "="*70)
print(" CONCLUSIONS")
print("="*70)

print("\n1. HUBBLE TENSION RESOLUTION")
print("-" * 50)
print(f"   Observed ΔH0: {observed_tension:.1f} km/s/Mpc")
print(f"   HRC predicted ΔH0: {hrc_tension:.1f} km/s/Mpc")
if abs(hrc_tension - observed_tension) < 2.0:
    print("   ✓ HRC CAN explain the Hubble tension")
else:
    print("   ✗ HRC does not fully explain the tension")

print("\n2. REQUIRED PARAMETERS")
print("-" * 50)
print(f"   ξ (non-minimal coupling): {hrc_params['xi']:.3f}")
print(f"   φ₀ (scalar field today): {hrc_params['phi_0']:.3f}")
print(f"   α (remnant coupling): {hrc_params['alpha']:.3f}")

print("\n3. OBSERVATIONAL CONSISTENCY")
print("-" * 50)
print(f"   BAO χ² (ΛCDM): {chi2_lcdm:.2f}")
print(f"   BAO χ² (HRC): {chi2_hrc:.2f}")

print("\n4. MODEL COMPARISON")
print("-" * 50)
print(f"   {bf_result['interpretation']}")

print("\n" + "="*70)